In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import networkx as nx
%matplotlib inline


In [40]:
import cvxpy as cp

In [3]:
plt.rcParams['figure.figsize'] = [15, 10]
sns.set_style('whitegrid')
plt.rcParams['font.size'] = 20.0
plt.rcParams['xtick.labelsize'] = 20.0
plt.rcParams['ytick.labelsize'] = 20.0
import pickle

# Generate synthetic data

In [31]:
n = 20
ground_truth_asymm = np.random.uniform(0, 0.5, (n, n))
ground_truth_mat = ground_truth_asymm + ground_truth_asymm.T

In [33]:
def gen_sample_mat(ground_truth_mat):
    '''
    Generate a sample matrix from a ground truth matrix
    inputs:
        ground_truth_mat: A n by n matrix of probabilities
    outputs:    
        A: a n by n matrix whose entries are 1 
        with probability ground_truth_mat[i, j] and 0 otherwise
    '''
    n = ground_truth_mat.shape[0]
    A = np.zeros_like(ground_truth_mat)
    for i in range(n):
        for j in range(i + 1):
            sample = np.random.binomial(1, ground_truth_mat[i, j])
            A[i, j] = sample
            A[j, i] = sample
    return A

In [35]:
sample_mats = [gen_sample_mat(ground_truth_mat) for _ in range(11)]

scale = np.sum(sample_mats[0])
for i in range(len(sample_mats)):
    if i > 0:
        s = np.sum(sample_mats[i])
        rescale = scale / s 
        sample_mats[i] = rescale * sample_mats[i]

In [36]:
for i in range(11): 
    print(np.sum(sample_mats[i]))

225.0
224.99999999999994
225.0
225.0
225.0
225.0
225.0
225.00000000000006
225.0
225.0
225.0


Notice that the sum of the entries of each matrix is the same, and is near 0.5 * 400. 

## Define the convex program

For now, we will just set delta manually to be O(sqrt(degree)). Here, the degree is roughly 10, so we set delta to be sqrt(10).

In [48]:
m = len(sample_mats)
n = sample_mats[0].shape[0]
delta = np.sqrt(10.0)

eta = cp.Variable(m, nonneg=True)
# partial_traces = cp.Variable(n)

# define the Y_diff matrix
X_0 = eta[0] * sample_mats[0]
X_arrs = []
for i in range(1, len(sample_mats)): 
    X_arrs.append(eta[i] * (X_0 - sample_mats[i]))
Y = X_0 + np.sum(X_arrs, axis=0)
Y_diff = Y - delta * np.eye(n)


partial_traces = []
for j in range(1, n + 1): 
    partial_trace = cp.atoms.lambda_sum_largest(Y_diff, j)
    partial_traces[j] = partial_trace

max_traces = cp.atoms.elementwise.maximum(partial_traces)
objective = cp.Minimize(max_traces)



TypeError: 'Variable' object does not support item assignment